<a href="https://colab.research.google.com/github/nelsonalbertohj/Magno-Parvo-CNN/blob/main/Data_analysis_and_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Analysis processes

In [2]:
!pip install tensorflow_addons

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import pandas as pd

import keras
import time
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Average
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
print("Tensorflow version: ",tf.__version__)

Tensorflow version:  2.8.0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
loc = "/content/drive/MyDrive/Vision Dataset/Imagenet-10-1500-splits/test"
data_dir = pathlib.Path(loc)

batch_size = 32
img_height = 224
img_width = 224

#Testing set
test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  label_mode = 'categorical',
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1297 files belonging to 10 classes.


In [6]:
def img_transforms(data,label):
    normalize_img = tf.keras.layers.Rescaling(1./255)
    color_norm = normalize_img(data)
    gray_img = tf.image.rgb_to_grayscale(data)
    gray_norm_img = normalize_img(gray_img)
    tfa.image.gaussian_filter2d(gray_norm_img,filter_shape=(10,10),sigma=5.0)
    concat_img = tf.concat([color_norm, gray_norm_img],axis=-1)
    return concat_img,label

AUTOTUNE = tf.data.AUTOTUNE
test_ds_preprocessed = test_ds.map(img_transforms)

CASHE_Test = "/content/drive/MyDrive/Vision Dataset/Imagenet-10-1500-splits/CASHE_Test_Blur_10_1500"
test_ds_preprocessed = test_ds_preprocessed.cache(CASHE_Test).prefetch(buffer_size=AUTOTUNE)

In [ ]:
# # Used to create CASHE in local disk by loading all images
# idx = 0
# start_time = time.time()
# for t in test_ds_preprocessed:
#   print("time to get object: ", time.time()-start_time)
#   idx += 1
#   start_time = time.time()

In [9]:
dual_stream_model = tf.keras.models.load_model("/content/drive/MyDrive/Vision Dataset/Models/Early_Testing_Models/Dualstream_with_ResNet_1000_Samples_Deeper_10Classes")
single_model = tf.keras.models.load_model("/content/drive/MyDrive/Vision Dataset/Models/Early_Testing_Models/Single_ResNet_1000_Samples_Deeper")

In [10]:
dual_test_loss, dual_test_accuracy = dual_stream_model.evaluate(test_ds_preprocessed)

single_test_loss, signle_test_accuracy = single_model.evaluate(test_ds_preprocessed)

41/41 [==============================] - 10s 214ms/step - loss: 1.0598 - accuracy: 0.7032


In [31]:
dual_pred_outputs = dual_stream_model.predict(test_ds_preprocessed)
dual_pred_outputs = np.argmax(dual_pred_outputs,axis=1)

single_pred_outputs = single_model.predict(test_ds_preprocessed)
single_pred_outputs = np.argmax(single_pred_outputs,axis=1)

true_outputs = np.concatenate([labels for _,labels in test_ds_preprocessed])
true_outputs = np.argmax(true_outputs,axis=1)

In [28]:
pred_outputs

array([2, 6, 3, ..., 3, 5, 9])

array([2, 6, 9, ..., 3, 9, 4])